In [1]:
# Se importa el set de datos y librerías necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# datos de pasos con 34 atributos a dataframe
datasetFull = pd.read_csv('datos_34.csv')  # carga CSV a data frame 

## A class is defined for the cromosomes (individuals with certain features)

In [3]:
class Cromosoma:
    '''
    Clase objeto cromosoma, con las propiedades de: cromosoma,
    fitness, número de atributos, dataset del cromosoma.
    Método: fit() para evaluar y darle valor al fitness del cromosoma
    '''
    # Inicialización de atributos
    def __init__(self, cromosoma, dataset):
        self.cromosoma = cromosoma
        self.n_atributos = 0
        for bit in self.cromosoma:
            if bit == '1':
                self.n_atributos+=1
        self.fitness = 0
        
        # Crear dataset (self.data) con solo los atributos del cromosoma
        counter = 0
        atributos = []
        for bit in self.cromosoma:
            counter+=1
            if bit == '0':
                string = 'Atributo_{}'.format(counter) 
                atributos.append(string)
        self.data = dataset.drop(columns=atributos)
            
    # Calcula fitness del cromosoma utilizando algoritmo de clasificación
    def fit(self):         
        
        # separar atributos en un data frame y etiquetas en otro
        X = self.data.drop(columns='class')
        Y = self.data['class']
        
        # Crear partición de entrenamiento y prueba de 80/20
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=69)
        
        # Clasificar con RandomForest
        clf = RandomForestClassifier(n_estimators=10, max_features=None, bootstrap=False, 
                             random_state=69) # 10 árboles en el bosque, usando todos los atributos y todos los datos
        
        clf.fit(X_train, Y_train)  # construye algoritmo con datos de entrenamiento
        self.fitness = clf.score(X_test, Y_test) # Accuracy promedio en datos de test y etiquetas
        
    # Apareo entre cromosoma y otro (mitades), con mutation rate de 0-1
    def breed(self, mate_cromosome, mutation_rate):
        offspring = []
        child = self.cromosoma[0:int(len(self.cromosoma)/2)]+mate_cromosome[int(len(self.cromosoma)/2):]
        for bit in child:
            offspring.append(bit)
            
        # crear mutaciones según tasa de mutación
        mutations = int(mutation_rate*len(self.cromosoma)) # número de mutaciones según tasa
        for i in range(mutations):
                        gene = random.randint(0, len(self.cromosoma)-1)
                        offspring[gene] = str(int(not(offspring[gene])))  # introducir mutación en bit random 
        return "".join(offspring)
    

foo = Cromosoma('1111111111111111111111111111111111', datasetFull)
foo.fit()
print("Accuracy utilizando los 34 atributos: {}".format(foo.fitness))

Accuracy utilizando los 34 atributos: 0.8677685950413223


### It starts generating an initial population of _n_ cromosomes with random features

In [5]:
import random, math

n = 10 # individuos por generación (se puede variar)
mutation_rate = 0.3  # tasa de mutaciones (se puede variar de 0 a 1)

poblacion = []
for i in range(n):
    cromo = ""
    for j in range(0, 34):
        bit = str(random.randint(0, 1))
        cromo+=bit
    poblacion.append(Cromosoma(cromo, datasetFull))

promedio = 0
for cromo in poblacion:
    print(cromo.cromosoma)
    cromo.fit()
    promedio+=cromo.fitness
    
print("Promedio: {} ".format(promedio/len(poblacion)))



1111111011011001100110111010110111
0000001110010111111011001110000000
0010100111011100101010110000010110
0010101101110100111101100010001100
0010011101111011100000111000001011
1000111101011010111101100010010110
0100000001110011101110111110001110
0111010011010010011110101101100011
1010101001011100101110001000100101
1101000111111110100101100111011011
Promedio: 0.7859504132231405 


### Genetic Algorithm Implementation

In [8]:
mejor_fitness = 0
generacion = 0
current_gen = poblacion

while (mejor_fitness<=0.91 and generacion<100): # condiciones de parada: fitness de 0.91 o 100 generaciones
    generacion+=1
    for cromosoma in current_gen:
        cromosoma.fit() # Realizar clasificación por medio de RandomForest
    current_gen = sorted(current_gen, reverse=True, key=lambda x:x.fitness)  # Reordenar en orden de mejor fitness
    mejor_fitness=current_gen[0].fitness
    
    # Hijos para generar 90% de siguiente generación
    next_gen = []
    for i in range(int(0.9*n)):
        # Parents are randomly chosen from the best half of the generation
        padre1 = random.choice(current_gen[:math.ceil(0.5*n)])  
        padre2 = random.choice(current_gen[:math.ceil(0.5*n)])
        hijo = padre1.breed(padre2.cromosoma, mutation_rate)
        
        # Meter cromosoma hijo solo si es válido (no son puros 0's)
        ones = 0
        for bit in hijo:
            if bit == '1':
                ones+=1
        if ones != 0:
            next_gen.append(Cromosoma(hijo, datasetFull))
        
    # Elitismo para generar individuos restantes de siguiente generación (con los mejores 10% )
    for i in range(len(current_gen)-len(next_gen)):
        next_gen.append(current_gen[i])
    
    print("Generación: {} \n Mejor fitness: {}".format(generacion, current_gen[0].fitness))
    print("Cantidad de cromosomas: {}".format(current_gen[0].n_atributos))
    current_gen = next_gen

# Reordenar conjunto final
current_gen = sorted(current_gen, reverse=True, key=lambda x:x.fitness)  


# TODOS
# cambiar cruce
# cambiar forma en que se hacen mutaciones

Generación: 1 
 Mejor fitness: 0.8347107438016529
Cantidad de cromosomas: 24
Generación: 2 
 Mejor fitness: 0.8347107438016529
Cantidad de cromosomas: 24
Generación: 3 
 Mejor fitness: 0.8429752066115702
Cantidad de cromosomas: 17
Generación: 4 
 Mejor fitness: 0.8677685950413223
Cantidad de cromosomas: 14
Generación: 5 
 Mejor fitness: 0.8677685950413223
Cantidad de cromosomas: 14
Generación: 6 
 Mejor fitness: 0.8677685950413223
Cantidad de cromosomas: 14
Generación: 7 
 Mejor fitness: 0.8677685950413223
Cantidad de cromosomas: 14
Generación: 8 
 Mejor fitness: 0.8677685950413223
Cantidad de cromosomas: 14
Generación: 9 
 Mejor fitness: 0.8677685950413223
Cantidad de cromosomas: 14
Generación: 10 
 Mejor fitness: 0.8677685950413223
Cantidad de cromosomas: 14
Generación: 11 
 Mejor fitness: 0.8677685950413223
Cantidad de cromosomas: 6
Generación: 12 
 Mejor fitness: 0.9008264462809917
Cantidad de cromosomas: 7
Generación: 13 
 Mejor fitness: 0.9090909090909091
Cantidad de cromosomas: 

### Print the final best individuals

In [7]:
for individuo in current_gen:
    individuo.fit()
    print("{}: Accuracy: {}".format(individuo.cromosoma, individuo.fitness))
    
    print("Número de atributos: {}".format(individuo.n_atributos))

current_gen[0].data.to_csv('best.csv', index=False)   
current_gen[0].data.head(10)

0001100000010001001011000000000000: Accuracy: 0.8925619834710744
Número de atributos: 7
0001100000010001000000000000000000: Accuracy: 0.8512396694214877
Número de atributos: 4
0000000000010001000000000000000000: Accuracy: 0.5619834710743802
Número de atributos: 2
0001100000000001000000000000000000: Accuracy: 0.7851239669421488
Número de atributos: 3
0001000000000001000000000000000000: Accuracy: 0.6115702479338843
Número de atributos: 2
0001100000010000000010000000000000: Accuracy: 0.8760330578512396
Número de atributos: 4
0000100000010000000001000000000000: Accuracy: 0.7768595041322314
Número de atributos: 3
0000100000010000001011000000000000: Accuracy: 0.7933884297520661
Número de atributos: 5
0001100000010000000000000000000000: Accuracy: 0.8016528925619835
Número de atributos: 3
0001100000000000000000000000000000: Accuracy: 0.8016528925619835
Número de atributos: 2


,Atributo_4,Atributo_5,Atributo_12,Atributo_16,Atributo_19,Atributo_21,Atributo_22,class
0,2.40,2.65,4.88,1.03,0.91,1.47,3.23,clase1
1,2.35,2.77,4.76,1.35,0.89,0.99,3.94,clase1
2,1.79,2.60,4.55,1.80,1.29,1.97,2.95,clase1
3,1.53,2.23,4.44,2.39,2.39,2.39,3.79,clase1
4,2.37,2.85,5.67,1.50,1.44,1.22,3.19,clase1
5,2.46,2.84,4.73,1.12,0.35,0.48,3.92,clase1
6,2.29,2.86,5.99,1.29,1.60,0.64,3.21,clase1
7,2.35,2.81,4.44,1.40,1.33,0.71,4.02,clase1
8,2.48,2.75,5.11,1.12,1.29,0.82,3.81,clase1
9,2.36,2.80,4.88,1.08,0.49,1.27,3.41,clase1
